In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .config('spark.ui.port', '4040') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 15:30:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import requests
import os
from pyspark.sql import types

In [3]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.LongType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.LongType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.LongType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.LongType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.LongType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.LongType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

In [13]:
import pandas as pd
import urllib

In [21]:
url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet'
input_path = 'green_tripdata_2021-02.parquet'
output_path = f'data/pq/green/2021/02/'
os.makedirs(output_path, exist_ok=True)
try:
    urllib.request.urlretrieve(url, input_path)
    print(f'done reading 2021-02 parquet')
except Exception as e:
    print(f"Error reading Parquet file from {url}: {e}")
print(f'start repartition and writing 2021-02 parquet')
df = spark.read.parquet(input_path, schema=green_schema)
print(df.dtypes)
df \
    .repartition(4) \
    .write.mode("overwrite").parquet(output_path)
os.remove(input_path)

done reading 2021-02 parquet
start repartition and writing 2021-02 parquet
[('VendorID', 'bigint'), ('lpep_pickup_datetime', 'timestamp_ntz'), ('lpep_dropoff_datetime', 'timestamp_ntz'), ('store_and_fwd_flag', 'string'), ('RatecodeID', 'double'), ('PULocationID', 'bigint'), ('DOLocationID', 'bigint'), ('passenger_count', 'double'), ('trip_distance', 'double'), ('fare_amount', 'double'), ('extra', 'double'), ('mta_tax', 'double'), ('tip_amount', 'double'), ('tolls_amount', 'double'), ('ehail_fee', 'int'), ('improvement_surcharge', 'double'), ('total_amount', 'double'), ('payment_type', 'double'), ('trip_type', 'double'), ('congestion_surcharge', 'double')]


In [22]:
year = 2020
taxi = 'green'
for month in range(1,13):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi}_tripdata_{year}-{month:02d}.parquet'
    input_path = f'{taxi}_tripdata_{year}-{month:02d}.parquet'
    output_path = f'data/pq/{taxi}/{year}/{month:02d}/'
    os.makedirs(output_path, exist_ok=True)
    try:
        urllib.request.urlretrieve(url, input_path)
        print(f'done reading {year}-{month}parquet')
    except Exception as e:
        print(f"Error reading Parquet file from {url}: {e}")
    print(f'start repartition and writing {year}-{month} parquet')
    df = spark.read.parquet(input_path, schema=green_schema)
    df \
        .repartition(4) \
        .write.mode("overwrite").parquet(output_path)
    print(f'finish repartition and writing {year}-{month} parquet')
    os.remove(input_path)
    print('pruge file')
    

done reading 2020-1parquet
start repartition and writing 2020-1 parquet
[('VendorID', 'bigint'), ('lpep_pickup_datetime', 'timestamp_ntz'), ('lpep_dropoff_datetime', 'timestamp_ntz'), ('store_and_fwd_flag', 'string'), ('RatecodeID', 'double'), ('PULocationID', 'bigint'), ('DOLocationID', 'bigint'), ('passenger_count', 'double'), ('trip_distance', 'double'), ('fare_amount', 'double'), ('extra', 'double'), ('mta_tax', 'double'), ('tip_amount', 'double'), ('tolls_amount', 'double'), ('ehail_fee', 'int'), ('improvement_surcharge', 'double'), ('total_amount', 'double'), ('payment_type', 'double'), ('trip_type', 'double'), ('congestion_surcharge', 'double')]


done reading 2020-2parquet
start repartition and writing 2020-2 parquet
[('VendorID', 'bigint'), ('lpep_pickup_datetime', 'timestamp_ntz'), ('lpep_dropoff_datetime', 'timestamp_ntz'), ('store_and_fwd_flag', 'string'), ('RatecodeID', 'double'), ('PULocationID', 'bigint'), ('DOLocationID', 'bigint'), ('passenger_count', 'double'), ('trip_distance', 'double'), ('fare_amount', 'double'), ('extra', 'double'), ('mta_tax', 'double'), ('tip_amount', 'double'), ('tolls_amount', 'double'), ('ehail_fee', 'int'), ('improvement_surcharge', 'double'), ('total_amount', 'double'), ('payment_type', 'double'), ('trip_type', 'double'), ('congestion_surcharge', 'double')]
done reading 2020-3parquet
start repartition and writing 2020-3 parquet
[('VendorID', 'bigint'), ('lpep_pickup_datetime', 'timestamp_ntz'), ('lpep_dropoff_datetime', 'timestamp_ntz'), ('store_and_fwd_flag', 'string'), ('RatecodeID', 'double'), ('PULocationID', 'bigint'), ('DOLocationID', 'bigint'), ('passenger_count', 'double'), ('trip_d

In [23]:
year = 2020
taxi = 'yellow'
for month in range(1,13):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi}_tripdata_{year}-{month:02d}.parquet'
    input_path = f'{taxi}_tripdata_{year}-{month:02d}.parquet'
    output_path = f'data/pq/{taxi}/{year}/{month:02d}/'
    os.makedirs(output_path, exist_ok=True)
    try:
        urllib.request.urlretrieve(url, input_path)
        print(f'done reading {year}-{month}parquet')
    except Exception as e:
        print(f"Error reading Parquet file from {url}: {e}")
    print(f'start repartition and writing {year}-{month} parquet')
    df = spark.read.parquet(input_path, schema=yellow_schema)
    df \
        .repartition(4) \
        .write.mode("overwrite").parquet(output_path)
    print(f'finish repartition and writing {year}-{month} parquet')
    os.remove(input_path)
    print('purge file')
    

done reading 2020-1parquet
start repartition and writing 2020-1 parquet


done reading 2020-2parquet
start repartition and writing 2020-2 parquet


done reading 2020-3parquet
start repartition and writing 2020-3 parquet


done reading 2020-4parquet
start repartition and writing 2020-4 parquet


done reading 2020-5parquet
start repartition and writing 2020-5 parquet
done reading 2020-6parquet
start repartition and writing 2020-6 parquet


...

done reading 2020-8parquet
start repartition and writing 2020-8 parquet


done reading 2020-9parquet
start repartition and writing 2020-9 parquet


done reading 2020-10parquet
start repartition and writing 2020-10 parquet


done reading 2020-11parquet
start repartition and writing 2020-11 parquet


done reading 2020-12parquet
start repartition and writing 2020-12 parquet


In [24]:
!ls -lh

total 48K
-rw-r--r-- 1 root root 1.6K Feb 25 21:21 Dockerfile
drwxr-xr-x 4 root root  128 Feb 27 15:57 data
drwxr-xr-x 4 root root  128 Feb 25 16:51 jupyter_notebook_docker
-rw-r--r-- 1 root root 1.5K Feb 25 21:13 pyspark_partition.py
-rw-r--r-- 1 root root   78 Feb 26 16:54 requirements.txt
-rw-r--r-- 1 root root  26K Feb 27 15:56 spark_groupby_join.ipynb
-rw-r--r-- 1 root root  13K Feb 25 15:35 taxi_zone_lookup.csv
-rw-r--r-- 1 root root  781 Feb 25 16:10 test_pyspark.py
-rw-r--r-- 1 root root 1.6K Feb 26 18:45 week5.rtf
drwxr-xr-x 6 root root  192 Feb 25 15:35 zones


In [25]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [26]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [27]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [28]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [29]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [30]:
from pyspark.sql import functions as F

In [31]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [32]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [33]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [34]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 1798748|
|      yellow|24649092|
+------------+--------+



In [41]:
df_trips_data.registerTempTable('trips_data')

In [42]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

[Stage 115:======================================>                  (6 + 2) / 9]

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 1798748|
|      yellow|24649092|
+------------+--------+



25/02/27 16:54:49 WARN Executor: Issue communicating with driver in heartbeater 
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1928

In [39]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

25/02/27 16:53:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [ ]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')